# Algorithms and Data Structures
Nathan Sharp | October 2020
****

# Lecture 4: Strassens Algorithm & The Master Theorem

## The Master Theorem
---

Useful in divide and conquer algorithms when you are splitting into $a$ subproblems all of size $n/b$.

Let $n_0 \in \mathbb{N}, k \in \mathbb{N}_0$  and $a,b \in \mathbb{R}$ with $a > 0$  and $b > 1$, and let $T: \mathbb{N} \rightarrow \mathbb{R}$ satisfy the following reccurence:

$$
T(n) =
  \begin{cases}
    \Theta (1)                     & \quad \text{if } n < n_0\\
    a \cdot T(n/b) + \Theta (n^k)  & \quad \text{if } n \geq n_0
  \end{cases}
$$

Let $c = \log_b(a)$; where we call $c$ the '_critical exponent_'. Then,

$$
T(n) =
\begin{cases}
  \Theta (n^c)                & \quad \text{if } k < c  & \quad \text{Case }I\\
  \Theta (n^c \cdot \log(n))  & \quad \text{if } k = c  & \quad \text{Case }II\\ 
  \Theta(n^k)                 & \quad \text{if } k > c  & \quad \text{Case }III
\end{cases}
$$

The Theorem also holds if we replace $a \cdot T(n/b)$ above with $a_1 \cdot T(\lfloor n/b \rfloor) + a_2 \cdot T(\lceil n/b \rceil)$ for any $a_1,a_2 \geq 0$ with $a_1 + a_2 = a$.

## Matrix Multiplication
---

In [2]:
def matMult(A,B):
    """Multiplies 2 square matrices"""             # Running Time
    n = len(A)                                     # O(1)
    C = [[j for j in range(n)] for i in range(n)]  # O(1)
    for i in range(n):                             # O(n)
        for j in range(n):                         # O(n)
            Cij = 0                                # O(1)
            for k in range(n):                     # O(n)
                Cij += A[i][k] * B[k][j]           # O(1) (significant step)
            C[i][j] = Cij                          # O(1)
    return C                                       # O(1)

# testing
A = [[1,2],[3,4]]
B = [[4,3],[2,1]]

matMult(A,B)

[[8, 5], [20, 13]]

### Running Time of naive matrix multiplication

$\underline{T(n) \in \Theta (n^3)}$

## Strassens Algorithm

Strassens Algorithm is based on a naive divide and conquer approach to matrix multiplication. It assumes n is a power of 2 (can be filled with 0's otherwise) noting,

$$
A = 
 \begin{pmatrix}
 \begin{array}{c|c}
  A_{11} & A_{12} \\
  \hline
  A_{21} & A_{22}  \\
 \end{array}
 \end{pmatrix}
$$

and

$$
B = 
 \begin{pmatrix}
 \begin{array}{c|c}
  B_{11} & B_{12} \\
  \hline
  B_{21} & B_{22}  \\
 \end{array}
 \end{pmatrix}
$$

Then, 

$$
C = AB = 
 \begin{pmatrix}
 \begin{array}{c|c}
  A_{11}B_{11} + A_{12}B_{21} & A_{11}B_{12} + A_{12}B_{22} \\
  \hline
  A_{21}B_{11} + A_{22}B_{21} & A_{21}B_{12} + A_{22}B_{22}  \\
 \end{array}
 \end{pmatrix}
$$


Note: this naive algorithm results in an identical $\Theta (n^3)$ runtime (infact it calculates the identical products).

Strassens algorithm uses a 'trick' in the classic divide and conquer appproach. It calculates the following 7 matrices;

$$
\begin{aligned}
& P_1 = (A_{11} + A_{22}) (B_{11} + B_{22})\\
& P_2 = (A_{21} + A_{22}) B_{11}\\ 
& P_3 = A_{11} (B_{12} - B_{22})\\
& P_4 = A_{22}(- B_{11} + B_{21})\\
& P_5 = (A_{11} + A_{12}) B_{22}\\
& P_6 = (- A_{11} + A_{12}) (B_{12}\\
& P_7 = (A_{11} - A_{22}) (B_{21} + B_{22})
\end{aligned}
$$

Which can be combined in the following ways to reach the result,

$$
\begin{aligned}
& C_{11} = P_1 + P_4 - P_5 + P_7\\
& C_{12} = P_3 + P_5\\
& C_{21} = P_2 + P_4\\
& C_{22} = P_1 + P_3 - P_2 + P_6\\
\end{aligned}
$$

Note: Strassens algorithm is only efficient for large matrices and will be outperformed by naive matrix multiplication for small inputs.

### Running time of Strassens Algorithm

- Splitting into the 7 submatrices recursivly requires $7 T(n/2) + \Theta (n^2)$
- Combining into quadrants to produce the final result takes $ 8(n/2)^2 = \Theta (n^2)$. (8 addition operations on matrices size $n/2$))


This gives the recurrence:
$$
\begin{aligned}
& T(n) = 7 T(n/2) + \Theta (n^2) + \Theta (n^2)\\ 
& T(n) = 7 T(n/2) + \Theta (n^2)\\
\end{aligned}
$$

Since $\log_2(7) = 2.807... (> 2)$, the Master Theorem gives us,

$$
\underline{T(n) = \Theta(n^{\log(7)})}
$$

## Implementation
---

In [4]:
import numpy as np

def partitionMatrix(matrix):
    length = len(matrix)
    if(length % 2 is not 0):
        stack = []
        for x in range(length + 1):
            stack.append(float(0))
        length += 1
        matrix = np.insert(matrix, len(matrix), values=0, axis=1)
        matrix = np.vstack([matrix, stack])
    d = (length // 2)
    matrix = matrix.reshape(length, length)
    completedPartition = [matrix[:d, :d], matrix[d:, :d], matrix[:d, d:], matrix[d:, d:]]
    return completedPartition

def strassen(mA, mB):
    n1 = len(mA)
    n2 = len(mB)
    global aN
    if(n1 and n2 <= aN):
        return (mA * mB)
    else:
        print(mA)
        A = partitionMatrix(mA)
        B = partitionMatrix(mB)
        mc = np.matrix([0 for i in range(len(mA))]for j in range(len(mB)))
        C = partitionMatrix(mc)


        a11 = np.array(A[0])
        a12 = np.array(A[2])
        a21 = np.array(A[1])
        a22 = np.array(A[3])

        b11 = np.array(B[0])
        b12 = np.array(B[2])
        b21 = np.array(B[1])
        b22 = np.array(B[3])

        mone = np.array(strassen((a11 + a22), (b11 + b22)))
        mtwo = np.array(strassen((a21 + a22), b11))
        mthree = np.array(strassen(a11, (b12 - b22)))
        mfour = np.array(strassen(a22, (b21 - b11)))
        mfive = np.array(strassen((a11 + a12), b22))
        msix = np.array(strassen((a21 - a11), (b11 + b12)))
        mseven = np.array(strassen((a12 - a22), (b21 + b22)))

        C[0] = np.array((mone + mfour - mfive + mseven))
        C[2] = np.array((mthree + mfive))
        C[1] = np.array((mtwo + mfour))
        C[3] = np.array((mone - mtwo + mthree + msix))

        return np.array(C)

    
# testing
A = [[1,2],[3,4]]
B = [[4,3],[2,1]]

strassen(A,B)

NameError: name 'aN' is not defined